In [1]:
from ToxicMl.trainer import GenericClassificationTrainer
from ToxicMl.metrics import Accuracy, F1, Recall, Precision
from ToxicMl.dataset import HivDataset
from torch_geometric.loader import ImbalancedSampler


from pathlib import Path

EPOCHS = 8

root = Path("dataset/ogbg_molhiv_custom")
dataset = HivDataset(root)
split_idx = dataset.get_idx_split() 
train = dataset[split_idx["train"]]
validation = dataset[split_idx["valid"]]
test = dataset[split_idx["test"]]

train_metrics = [F1(), Accuracy(), Recall(), Precision()]
validation_metrics = [F1(), Accuracy(), Recall(), Precision()]

In [2]:
from ToxicMl.MLmodels.attention import ChemAttentionBlock
import torch
import torch.nn.functional as F

from torch.nn import Linear, ReLU
from torch_geometric.data import Data
from torch_geometric.nn.pool import global_max_pool

from torch.nn import BatchNorm1d

class ChemAttentionDescriptors(torch.nn.Module):
    def __init__(self, embedder_depth, in_channels, hidden_channels, out_channels, in_descriptors):
        super().__init__()
        self.in_descriptors = in_descriptors
        self.encoder = torch.nn.ModuleList([
            ChemAttentionBlock(in_channels=in_channels, out_channels=hidden_channels)
            if i == 0
            else ChemAttentionBlock(in_channels=hidden_channels, out_channels=hidden_channels)
            for i in range(embedder_depth)])
        self.norm1 = BatchNorm1d(hidden_channels+in_descriptors)
        self.linear1 = Linear(hidden_channels+in_descriptors, 64)
        self.relu1 = ReLU(inplace=True)
        self.linear2 = Linear(64, 32)
        self.relu2 = ReLU(inplace=True)
        self.linear3 = Linear(32, out_channels)
        self.relu3 = ReLU(inplace=True)
        

    def forward(self, data: Data):
        for module in self.encoder:
            data = module(data)
        x = global_max_pool(data.x, data.batch)


        d = data.descriptors.view(-1, self.in_descriptors)
        x = torch.cat([x,d], dim=1)
        x = self.norm1(x)
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        return F.softmax(x)
    

In [3]:
from ToxicMl.MLmodels.attention import ChemAttention
model = ChemAttentionDescriptors(3, 133, 16, 2, 202)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.CrossEntropyLoss()
sampler = ImbalancedSampler(train) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericClassificationTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    sampler,
    EPOCHS,
    None,
    32
)

trainer.train("HIV ChemAttention 3-16, sampling, custom dataset, descriptors")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: cus-tibor (cus-tibor-none). Use `wandb login --relogin` to force relogin


train epoch:   0%|          | 0/1029.0 [00:00<?, ?it/s]/tmp/ipykernel_14768/3053635579.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
train epoch: 100%|██████████| 1029/1029.0 [00:09<00:00, 106.75it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 116.10it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:09<00:00, 103.99it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 213.94it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:10<00:00, 101.41it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 203.58it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:11<00:00, 89.72it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 182.57it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:11<00:00, 86.15it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 170.45it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:13<00:00, 73.64it/s]
val epoch: 100%|██████████| 129/129.0 

train/Accuracy,▁▇▇██▇██
train/F1,▁▇▇▇█▇██
train/Precision,▁▄▆▇▇▆▇█
train/Recall,▁█▇██▇██
train/loss,▆▁▂▄▄▆█▇
validation/Accuracy,▂▇█▅▆▄▆▁
validation/F1,▂▇█▅▆▄▆▁
validation/Precision,▁██▁▁█▁▁
validation/Recall,▂▇█▅▆▄▆▁
validation/loss,▅▄█▃▁▃▆▃
train/Accuracy,0.789


In [4]:
model = ChemAttentionDescriptors(3, 133, 32, 2, 202)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.CrossEntropyLoss()
sampler = ImbalancedSampler(train) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericClassificationTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    sampler,
    EPOCHS,
    None,
    32
)

trainer.train("HIV ChemAttention 3-32, sampling, custom dataset, descriptors")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/1029.0 [00:00<?, ?it/s]/tmp/ipykernel_14768/3053635579.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
train epoch: 100%|██████████| 1029/1029.0 [00:11<00:00, 89.94it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 167.24it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:13<00:00, 76.41it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 148.07it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:13<00:00, 77.29it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 171.98it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:12<00:00, 79.89it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 151.33it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:12<00:00, 81.18it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 137.54it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:14<00:00, 73.09it/s]
val epoch: 100%|██████████| 129/129.0 [00

train/Accuracy,▁▇▇▇████
train/F1,▁▇▇▇████
train/Precision,▁▇▇▇██▇█
train/Recall,▁▇█████▇
train/loss,▅▄▂▃▂█▅▁
validation/Accuracy,▂█▁▄▆▃▃▂
validation/F1,▄█▁▇▇▃▃▃
validation/Precision,█▁▅█▅▃▅▆
validation/Recall,▄█▁▆▇▃▃▃
validation/loss,▅▁▅▂█▂▅▁
train/Accuracy,0.8114


In [5]:
from ToxicMl.MLmodels.attention import ChemAttention
model = ChemAttentionDescriptors(3, 133, 64, 2, 202)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.CrossEntropyLoss()
sampler = ImbalancedSampler(train) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericClassificationTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    sampler,
    EPOCHS,
    None,
    32
)

trainer.train("HIV ChemAttention 3-64, sampling, custom dataset, descriptors")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/1029.0 [00:00<?, ?it/s]/tmp/ipykernel_14768/3053635579.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
train epoch: 100%|██████████| 1029/1029.0 [00:15<00:00, 67.80it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 147.93it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:14<00:00, 70.32it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 168.98it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:16<00:00, 62.51it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 135.44it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:16<00:00, 63.70it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 146.50it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:15<00:00, 65.53it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 126.31it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:14<00:00, 69.16it/s]
val epoch: 100%|██████████| 129/129.0 [00

train/Accuracy,▁▇▇█████
train/F1,▁▇▇█████
train/Precision,▁▇██████
train/Recall,▁▇▇█████
train/loss,▁█▆▅▃▂▆▄
validation/Accuracy,▃▅█▇█▇▅▁
validation/F1,▂▄█▃█▆▅▁
validation/Precision,▆▅▅▁▅▅▆█
validation/Recall,▂▄█▄█▆▅▁
validation/loss,▅▁▃▃▄▅█▂
train/Accuracy,0.8015


In [6]:
from ToxicMl.MLmodels.attention import ChemAttention
model = ChemAttentionDescriptors(5, 133, 16, 2, 202)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.CrossEntropyLoss()
sampler = ImbalancedSampler(train) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericClassificationTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    sampler,
    EPOCHS,
    None,
    32
)

trainer.train("HIV ChemAttention 5-16, sampling, custom dataset, descriptors")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/1029.0 [00:00<?, ?it/s]/tmp/ipykernel_14768/3053635579.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
train epoch: 100%|██████████| 1029/1029.0 [00:13<00:00, 75.64it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 141.40it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:15<00:00, 67.10it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 148.96it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:15<00:00, 67.20it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 145.36it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:15<00:00, 68.37it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 144.13it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:15<00:00, 67.84it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 152.25it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:14<00:00, 68.89it/s]
val epoch: 100%|██████████| 129/129.0 [00

train/Accuracy,▁▇███▇██
train/F1,▁▇███▇██
train/Precision,▁▆█▇▇▇█▇
train/Recall,▁▆▇▇█▆▇▇
train/loss,▂▃▁▄▃▃█▁
validation/Accuracy,█▁▅▇█▄▅▇
validation/F1,█▁▃▇▇▃▃█
validation/Precision,▃█▁▃▁▅▁▅
validation/Recall,█▁▃▇▇▃▃█
validation/loss,▇▂▆▃█▂▂▁
train/Accuracy,0.8107


In [7]:
from ToxicMl.MLmodels.attention import ChemAttention
model = ChemAttentionDescriptors(5, 133, 32, 2, 202)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.CrossEntropyLoss()
sampler = ImbalancedSampler(train) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericClassificationTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    sampler,
    EPOCHS,
    None,
    32
)

trainer.train("HIV ChemAttention 5-32, sampling, custom dataset, descriptors")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/1029.0 [00:00<?, ?it/s]/tmp/ipykernel_14768/3053635579.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
train epoch: 100%|██████████| 1029/1029.0 [00:15<00:00, 66.45it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 137.46it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:16<00:00, 61.06it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 137.47it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:17<00:00, 60.51it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 143.42it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:16<00:00, 62.50it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 136.22it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:17<00:00, 59.62it/s]
val epoch: 100%|██████████| 129/129.0 [00:00<00:00, 129.97it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:19<00:00, 51.71it/s]
val epoch: 100%|██████████| 129/129.0 [00

train/Accuracy,▁▇█▇▇█▇█
train/F1,▁▇█▇▇█▇█
train/Precision,▁▇█▇▇███
train/Recall,▁▇█▇▇▇▇█
train/loss,▂▁█▆▄▄▁▂
validation/Accuracy,▁█▆▆▇▇▇▇
validation/F1,▁█▇▅▇▆▆▇
validation/Precision,█▁▅▂▂▂▁▃
validation/Recall,▁█▇▅▇▆▆▇
validation/loss,█▁▅▄▆█▇█
train/Accuracy,0.8103


In [8]:
from ToxicMl.MLmodels.attention import ChemAttention
model = ChemAttentionDescriptors(5, 133, 64, 2, 202)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.CrossEntropyLoss()
sampler = ImbalancedSampler(train) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericClassificationTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    sampler,
    EPOCHS,
    None,
    32
)

trainer.train("HIV ChemAttention 5-64, sampling, custom dataset, descriptors")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/1029.0 [00:00<?, ?it/s]/tmp/ipykernel_14768/3053635579.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
train epoch: 100%|██████████| 1029/1029.0 [00:24<00:00, 42.84it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 110.12it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:22<00:00, 46.13it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 97.62it/s] 
train epoch: 100%|██████████| 1029/1029.0 [00:22<00:00, 45.53it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 105.03it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:22<00:00, 44.86it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 109.31it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:21<00:00, 47.38it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 115.88it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:21<00:00, 47.20it/s]
val epoch: 100%|██████████| 129/129.0 [00

train/Accuracy,▁▇██████
train/F1,▁▇██████
train/Precision,▁▇██████
train/Recall,▁▇██▇█▇█
train/loss,▅▁█▂▄▆▁▄
validation/Accuracy,▁▅▇▃█▁▆▄
validation/F1,▁▄▅▅▇▂█▃
validation/Precision,▄▂▁█▃▅▆▄
validation/Recall,▁▄▆▅█▂█▃
validation/loss,█▃▄▄▄▁▄▅
train/Accuracy,0.7982


In [9]:
from ToxicMl.MLmodels.attention import ChemAttention
model = ChemAttentionDescriptors(5, 133, 128, 2, 202)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.09)
loss_fn = torch.nn.CrossEntropyLoss()
sampler = ImbalancedSampler(train) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainer = GenericClassificationTrainer(
    model,
    optimizer,
    loss_fn,
    lr_scheduler,
    train_metrics,
    validation_metrics,
    train,
    validation,
    test,
    device,
    sampler,
    EPOCHS,
    None,
    32
)

trainer.train("HIV ChemAttention 5-128, sampling, custom dataset, descriptors")

/home/tibor/Documents/msc-datascience/2024w/Toxic/ToxicML/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train epoch:   0%|          | 0/1029.0 [00:00<?, ?it/s]/tmp/ipykernel_14768/3053635579.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
train epoch: 100%|██████████| 1029/1029.0 [00:28<00:00, 36.23it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 77.20it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:30<00:00, 33.83it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 85.39it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:30<00:00, 34.25it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 88.63it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:30<00:00, 33.43it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 72.29it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:31<00:00, 32.35it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<00:00, 77.50it/s]
train epoch: 100%|██████████| 1029/1029.0 [00:30<00:00, 34.10it/s]
val epoch: 100%|██████████| 129/129.0 [00:01<0

train/Accuracy,▁▇█▇████
train/F1,▁▇█▇████
train/Precision,▁▇█▇█▇▇█
train/Recall,▁▇▇▇███▇
train/loss,▇▅▄▁▁█▅█
validation/Accuracy,▂█▄▁▂▅█▅
validation/F1,▁▇▅▆▃▅█▄
validation/Precision,▁▄▄█▃▄▅▃
validation/Recall,▁▇▄▅▃▅█▄
validation/loss,▄▂▄▆▃▂▁█
train/Accuracy,0.7923
